In [40]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchinfo import summary
import modules.data_setup as data_setup
from modules.engine import train_step,test_step,train
import os


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


In [41]:
version = torch.__version__
print(f"PyTorch Version: {version}")

PyTorch Version: 2.8.0


In [42]:
import requests
from pathlib import Path
import zipfile

data_path = Path('data')
image_path = data_path / 'pizza_steak_sushi_20_percent'

if image_path.is_dir():
    print(f'{image_path} exists')
else:
    print(f'{image_path} does not exist, creating...')
    image_path.mkdir(parents=True, exist_ok=True)

    with open(data_path / 'pizza_steak_sushi_20_percent.zip','wb') as f:
         request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip")
         print('Downloading pizza_steak_sushi_20_percent data')
         f.write(request.content)
    with zipfile.ZipFile(data_path / 'pizza_steak_sushi_20_percent.zip','r') as zip_ref:
        print("Extracting pizza_steak_sushi_20_percent.zip")
        zip_ref.extractall(image_path)

    os.remove(data_path / 'pizza_steak_sushi_20_percent.zip')

data/pizza_steak_sushi_20_percent exists


In [43]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #device agnostic setting

In [44]:
def set_seed(seed:int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [45]:
train_dir = image_path/'train'
test_dir = image_path/'test'
normalize = transforms.Normalize(
    mean=[0.485,0.456,0.406],
    std=[0.229,0.224,0.225]
)
simple_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])
print(simple_transform)


Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [46]:
data_augmentation_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    normalize
])
print(data_augmentation_transform)

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    TrivialAugmentWide(num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [47]:
train_dataloader,test_dataloader,class_names = data_setup.create_dataloaders(
                                                train_dir=train_dir,
                                                test_dir=test_dir,
                                                train_transform=simple_transform,
                                                test_transform=simple_transform,
                                                batch_size=32)
train_dataloader,test_dataloader,class_names

(<torch.utils.data.dataloader.DataLoader at 0x16b9c5430>,
 ['pizza', 'steak', 'sushi'])

In [48]:
train_dataloader_with_aug,test_dataloader,class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    train_transform=data_augmentation_transform,
    test_transform=simple_transform,
    batch_size=32
)
train_dataloader_with_aug,test_dataloader,class_names

(<torch.utils.data.dataloader.DataLoader at 0x16b90d160>,
 ['pizza', 'steak', 'sushi'])

In [49]:
output_shape = len(class_names)
def create_effnetb2():

    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    model = torchvision.models.efficientnet_b2(weights=weights)

    for param in model.parameters():
        param.requires_grad = False

    set_seed()

    model_classifier = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features=1408,out_features=output_shape)
    )

    model.name = 'effnetb2'
    print(f'[INFO] efficientnet-b2 model created: {model.name}')
    return model

In [50]:
num_epochs = [5,10]
effnetb2 = create_effnetb2()

[INFO] efficientnet-b2 model created: effnetb2


In [51]:
models = ['effnetb2']
train_dataloaders = {
    'data_without_augmentation': train_dataloader,
    'data_with_augmentation': train_dataloader_with_aug,
}

In [52]:
from modules.save_model import save_model
from modules.engine import train
from modules.create_writer import create_writer
from torch.utils.tensorboard.writer import SummaryWriter
set_seed(42)

experiment_number = 0

for dataloader_name,train_dataloader in train_dataloaders.items():
    for epochs in num_epochs:
        for model_name in models:
            experiment_number += 1
            print(f'[INFO] Experiment number: {experiment_number}')
            print(f'[INFO] Model name: {model_name}')
            print(f'[INFO] Dataloader:{dataloader_name}')
            print(f'[INFO] Epoch number: {epochs}')

            if model_name == 'effnetb2':
                model = create_effnetb2()

            loss_fn = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

            train(
                model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                loss_fn=loss_fn,
                optimizer=optimizer,
                epochs=epochs,
                device=device,
                writer=create_writer(
                    experiment_name=dataloader_name,
                    model_name=model_name,
                    extra=f"{epochs}_epochs"
                )
            )

            save_filepath = f'{model_name}_{dataloader_name}_{epochs}_epochs.pth'
            save_model(
                model=model,
                target_dir="models",
                model_name=save_filepath,
            )
            print("-"*50 + "\n")

[INFO] Experiment number: 1
[INFO] Model name: effnetb2
[INFO] Dataloader:data_without_augmentation
[INFO] Epoch number: 5
[INFO] efficientnet-b2 model created: effnetb2
[INFO] Created SummaryWriter. saving to runs/2025-09-28, 14-08-17/data_without_augmentation/effnetb2/5_epochs...


  0%|          | 0/5 [00:19<?, ?it/s]


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn